# Independence of mechanism

The idea is that the parameters of causal Markov kernel for a node should be independent of that nodes causes

In [12]:
import torch
import pyro
import pyro.distributions as dist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyro.poutine import trace
from pprint import pprint
#from matplotlib.pyplot import hist
#true model
def true_model(n):
    alpha = pyro.sample('alpha', dist.Normal(2.0, 1.0))
    beta = pyro.sample('beta', dist.Normal(3.0,1.0))
    with pyro.plate('data', n):
        X = pyro.sample('X', dist.Normal(alpha,1.0))
        Y_mean = beta*(X**2)+1
        Y = pyro.sample('Y', dist.Normal(Y_mean,1.0))
    return X,Y
true_model(10)

(tensor([1.1045]), tensor([4.8507]))

In [18]:
tr = trace(true_model(1))
tr.trace

In [13]:
#uncon_true_model = [true_model() for i in range(500)]
#X_data, Y_data = zip(*uncon_true_model)
traces = []
for _ in range(1000):
    tr = trace(true_model(1)).get_trace()
    values = {
        name: props['value'].item()
        for (name, props) in tr.nodes.items()
        if props['type'] == 'sample'
    }
    traces.append(values)

pd.DataFrame(traces).hist()


TypeError: 'tuple' object is not callable

We want to calculate $P(alpha|X,Y)$ and P(beta|X,Y)

In [25]:
#I don't know how to write the condition model!!!! pass entire data
x_data, y_data = true_model(1000)
conditioned_true_model = pyro.condition(true_model, data = {"X":x_data, "Y":y_data})
#conditioned_true_model(10)

In [48]:
def guide(n):
    a = pyro.param('a',torch.tensor(2.0))
    b = pyro.param('b',torch.tensor(3.0))
    alpha = pyro.sample('alpha', dist.Normal(a, 1.0))
    beta = pyro.sample('beta', dist.Normal(b,1.0))
    
from pyro.infer import SVI, Trace_ELBO

svi = SVI(conditioned_true_model, guide, pyro.optim.Adam({"lr":0.05}),loss = Trace_ELBO())
pyro.clear_param_store()
for i in range(5000):
    elbo = svi.step(1000)

 

In [49]:
pyro.param("a")


tensor(1.7386, requires_grad=True)

In [50]:
pyro.param("b")

tensor(1.8836, requires_grad=True)

In [26]:
from pyro.infer.mcmc import MCMC
from pyro.infer.mcmc.nuts import HMC
from pyro.infer import EmpiricalMarginal
import matplotlib.pyplot as plt

n = 1000
alpha_prior = 2.
hmc_kernel = HMC(conditioned_true_model, step_size=0.9, num_steps=4)
posterior = MCMC(hmc_kernel, 
                 num_samples=1000, 
                 warmup_steps=50)
posterior.run(n) #gives list of tuples of alpha and beta
posterior.get_samples() #dic keys are alpha and beta

Sample: 100%|██████████| 1050/1050 [03:04,  5.69it/s, step size=4.83e-02, acc. prob=0.944]


{'alpha': tensor([1.4202, 1.4347, 1.3634, 1.4358, 1.4087, 1.4031, 1.4315, 1.4258, 1.3915,
         1.4322, 1.3412, 1.4590, 1.3853, 1.3853, 1.4168, 1.4024, 1.3175, 1.4518,
         1.3533, 1.4130, 1.3632, 1.4105, 1.4087, 1.3744, 1.3952, 1.3938, 1.4370,
         1.3842, 1.3515, 1.4090, 1.4297, 1.3892, 1.4069, 1.4153, 1.3854, 1.4247,
         1.4247, 1.3962, 1.4065, 1.4420, 1.3915, 1.3453, 1.4551, 1.3881, 1.4179,
         1.3531, 1.4053, 1.3731, 1.4724, 1.3789, 1.4180, 1.3947, 1.4068, 1.4186,
         1.3747, 1.4280, 1.3569, 1.4137, 1.4921, 1.3890, 1.4041, 1.4082, 1.3820,
         1.3961, 1.3897, 1.4583, 1.4198, 1.3960, 1.3964, 1.4122, 1.3916, 1.3763,
         1.4492, 1.3957, 1.4556, 1.3672, 1.3672, 1.4293, 1.3569, 1.4719, 1.4238,
         1.4290, 1.4205, 1.4330, 1.3923, 1.4373, 1.4412, 1.4209, 1.3795, 1.3564,
         1.4724, 1.3524, 1.4679, 1.4203, 1.3748, 1.3997, 1.4143, 1.4071, 1.3816,
         1.4010, 1.4049, 1.3933, 1.3569, 1.4195, 1.4367, 1.4226, 1.4226, 1.3736,
         1.4155, 1.

In [24]:
posterior.get_samples()['beta'].mean()

tensor(2.1285)